In [ ]:
import requests
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
API_KEY = '2e3a785a0323fb22d89acae1264b47cb449782aa'

#Making API requests
get = 'GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F'
company_url = f'https://api.census.gov/data/2018/abscs?get={get}&for=us:*&key={API_KEY}'

get = 'GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,PAYANN_PCT_S_F'
business_url = f'https://api.census.gov/data/2018/abscb?get={get}&for=us:*&QDESC_LABEL=SPOUSES&key={API_KEY}'

get = 'GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F,OWNPDEMP_S,OWNPDEMP_S_F,OWNPDEMP_PCT_S,OWNPDEMP_PCT_S_F'
owner_url = f'https://api.census.gov/data/2018/abscbo?get={get}&for=us:*&QDESC_LABEL=YRACQBUS&key={API_KEY}'

company_request = requests.get(company_url)
business_request = requests.get(business_url)
owner_request = requests.get(owner_url)

In [20]:
def to_dataframe(data):
    '''Takes census raw data, cleans it and places it into a pandas dataframe
    which is returned to the user'''

    # Remove brackets from ends
    data = data[2:-2]
    # Remove quotes from values
    data = data.replace('"', '')
    # Change NAICS2017_LABEL value names containing commas
    data = data.replace('Agriculture, forestry, fishing and hunting', 'Agriculture/Forestry/Fishing/Hunting')
    data = data.replace('Mining, quarrying, and oil and gas extraction', 'Mining/Quarrying/Oil&Gas Extraction')
    data = data.replace('Professional, scientific, and technical services', 'Professional/Scientific/Technical Services')
    data = data.replace('Arts, entertainment, and recreation', 'Arts/Entertainment/Recreation')
    # Splitting into rows
    data = data.split('],\n[')
    # Changing into a list to seperate headers and values
    new_data = [i.split(',') for i in data]

    # Creating DataFrame
    values = new_data[1:]
    headers = new_data[0]
    df = pd.DataFrame(values, columns = headers)
    return df

In [21]:
company_data = company_request.text
company_summary = to_dataframe(company_data)
company_summary.head()

# company_data = company_request.text
# Excluding brackets
# company_data = company_data[2:-2]
#Removing " from entries
# company_data = company_data.replace('"', '')

# Renamed NAICS groups containing commas
# company_data = company_data.replace('Agriculture, forestry, fishing and hunting', 'Agriculture/Forestry/Fishing/Hunting')
# company_data = company_data.replace('Mining, quarrying, and oil and gas extraction', 'Mining/Quarrying/Oil&Gas Extraction')
# company_data = company_data.replace('Professional, scientific, and technical services', 'Professional/Scientific/Technical Services')
# company_data = company_data.replace('Arts, entertainment, and recreation', 'Arts/Entertainment/Recreation')
# company_data = company_data.split('],\n[')

# # new_data = [i.split(',') for i in company_data]
# company_data = new_data[1:]
# columns = new_data[0]

# census_company = pd.DataFrame(company_data, columns=columns)

,GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,EMPSZFI,EMPSZFI_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,RCPPDEMP,RCPPDEMP_F,EMP,EMP_F,PAYANN,PAYANN_F,FIRMPDEMP_S,FIRMPDEMP_S_F,RCPPDEMP_S,RCPPDEMP_S_F,EMP_S,EMP_S_F,PAYANN_S,PAYANN_S_F,us
0,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,001,Total,001,All firms,2018,84496,null,91103725,null,722654,null,21975675,null,3.0,null,8.9,null,7.7,null,7.0,null,1
1,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,001,Total,611,Firms with no employees,2018,10036,null,3233978,null,0,null,499996,null,4.7,null,19.2,null,0.0,null,13.5,null,1
2,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,001,Total,612,Firms with 1 to 4 employees,2018,47105,null,15792524,null,92631,null,2808773,null,4.7,null,8.2,null,4.8,null,7.4,null,1
3,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,001,Total,620,Firms with 5 to 9 employees,2018,12523,null,10688294,null,83449,null,2601817,null,5.9,null,4.3,null,5.6,null,6.3,null,1
4,0100000US,United States,00,Total for all sectors,002,Female,020,Hispanic,00,Total,001,Total,630,Firms with 10 to 19 employees,2018,8516,null,14105603,null,116479,null,3506384,null,6.0,null,16.2,null,6.7,null,10.7,null,1


In [22]:
business_data = business_request.text
businesschar = to_dataframe(business_data)
businesschar.head()

# business_data = business_request.text
# business_data = business_data[2:-2]
# business_data = business_data.replace('"', '')
# business_data = business_data.replace('Agriculture, forestry, fishing and hunting', 'Agriculture/Forestry/Fishing/Hunting')
# business_data = business_data.replace('Mining, quarrying, and oil and gas extraction', 'Mining/Quarrying/Oil&Gas Extraction')
# business_data = business_data.replace('Professional, scientific, and technical services', 'Professional/Scientific/Technical Services')
# business_data = business_data.replace('Arts, entertainment, and recreation', 'Arts/Entertainment/Recreation')
# business_data = business_data.split('],\n[')
# new_data = [i.split(',') for i in business_data]

# business_data = new_data[1:]
# headers = new_data[0]
# businesschar = pd.DataFrame(business_data, columns=headers)

,GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,PAYANN_PCT_S_F,QDESC_LABEL,us
0,0100000US,United States,00,Total for all sectors,001,Total,001,Total,00,Total,001,Total,B03,SPOUSES,A1,All firms,2018,5722142,null,0.0,X,38405539368,null,0.0,X,128196406,null,0.0,X,6937240308,null,0.0,X,0.0,null,0.0,X,1.2,null,0.0,X,0.5,null,0.0,X,0.5,null,0.0,X,SPOUSES,1
1,0100000US,United States,00,Total for all sectors,001,Total,001,Total,00,Total,001,Total,B03,SPOUSES,LZ,Jointly owned and equally operated by spouses,2018,205568,null,8.3,null,283097669,null,6.5,null,1872957,null,9.0,null,62359539,null,7.0,null,1.5,null,0.1,null,4.5,null,0.4,null,4.8,null,0.5,null,4.3,null,0.4,null,SPOUSES,1
2,0100000US,United States,00,Total for all sectors,001,Total,001,Total,00,Total,001,Total,B03,SPOUSES,MA,Jointly owned but primarily operated by male s...,2018,147635,null,6.0,null,280871091,null,6.5,null,1260228,null,6.1,null,46391946,null,5.2,null,1.4,null,0.1,null,16.2,null,1.0,null,8.3,null,0.5,null,5.8,null,0.3,null,SPOUSES,1
3,0100000US,United States,00,Total for all sectors,001,Total,001,Total,00,Total,001,Total,B03,SPOUSES,MB,Jointly owned but primarily operated by female...,2018,47766,null,1.9,null,44638710,null,1.0,null,359141,null,1.7,null,10960953,null,1.2,null,3.2,null,0.1,null,10.1,null,0.1,null,6.7,null,0.1,null,7.8,null,0.1,null,SPOUSES,1
4,0100000US,United States,00,Total for all sectors,001,Total,001,Total,00,Total,001,Total,B03,SPOUSES,MC,Not jointly owned by spouses,2018,2072921,null,83.8,null,3732535972,null,86.0,null,17211864,null,83.1,null,769528253,null,86.5,null,0.3,null,0.2,null,1.5,null,0.8,null,0.8,null,0.9,null,1.7,null,0.6,null,SPOUSES,1


In [23]:
owner_data = owner_request.text
ownerschar = to_dataframe(owner_data)
ownerschar.head()

,GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F,OWNPDEMP_S,OWNPDEMP_S_F,OWNPDEMP_PCT_S,OWNPDEMP_PCT_S_F,QDESC_LABEL,us
0,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,001,All owners of respondent firms,O02,YRACQBUS,CI03,2013 to 2017,2018,21939,null,29.9,null,5.1,null,1.2,null,YRACQBUS,1
1,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,001,All owners of respondent firms,O02,YRACQBUS,CM03,2018,2018,912,null,1.2,null,20.3,null,0.3,null,YRACQBUS,1
2,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,001,All owners of respondent firms,O02,YRACQBUS,CN,Don't know,2018,7441,null,10.1,null,4.2,null,0.5,null,YRACQBUS,1
3,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,001,All owners of respondent firms,O02,YRACQBUS,CO,Total reporting,2018,73427,null,100.0,null,2.1,null,0.0,null,YRACQBUS,1
4,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,001,All owners of respondent firms,O02,YRACQBUS,CP,Item not reported,2018,3106,null,0.0,X,5.2,null,0.0,X,YRACQBUS,1
